# 追加の週末運動 - 第2週

2週目から学んだことすべてを使用して、1週目のエクササイズで作成した技術的な質問/回答者の完全なプロトタイプを作成します。

これには、Gradio UI、ストリーミング、専門知識を追加するためのシステムプロンプトの使用、およびモデル間を切り替える機能が含まれます。ツールの使用を実証できる場合は、ボーナスポイント！

大胆だと感じた場合は、オーディオ入力を追加できるかどうかを確認して、それに相談して、オーディオで応答してもらいます。 ChatGptまたはClaudeはあなたを助けることができます。

私はすぐにここに完全なソリューションを公開します - 誰かが私をそれにbeatっていない限り...

言語の家庭教師から、会社のオンボーディングソリューション、コンパニオンAIまで、これには非常に多くの商用アプリケーションがあります（このようなもの！）私はあなたの結果を見るのを待ちきれません。

---

このノートブックは「航空会社AIアシスタント」のプロトタイプを作成する演習です。主な処理内容は以下の通りです。

https://github.com/ed-donner/llm_engineering/blob/main/week2/community-contributions/week2-EXERCISE-booking-translation-audio_input-history_audio.ipynb

### 機能概要
- チャットUI（Gradio）を使ったAIアシスタントとの対話
- 航空券の価格取得や予約ができるツール機能
- ユーザーやAIの発話を音声合成（TTS）で再生
- 入出力メッセージの自動翻訳（多言語対応）
- マイクからの音声入力→文字起こし（Whisper利用）
- チャット履歴や翻訳済み履歴の管理
- モデルの切り替え、システムプロンプトによる応答制御

### 技術要素
- OpenAI API（GPT, TTS, Whisper）
- deep_translator（Google翻訳API）
- Gradio（Web UI構築）
- 音声ファイル再生（pydub, IPython.display）

### 具体的な処理の流れ
1. 必要なライブラリのインストール（deep_translator, openai-whisper, soundfileなど）
2. OpenAI APIキーの環境変数から取得
3. 航空券の価格取得・予約を行う関数を定義し、ツールとして登録
4. TTSでの音声応答、音声入力（Whisper）などのユーティリティ関数を実装
5. deep_translatorで多言語翻訳機能を実装
6. チャット履歴・翻訳履歴の管理＆UIのドロップダウンで再生メッセージ選択
7. 音声入力された内容を文字起こししてチャット履歴に追加
8. モデル推論結果やツール呼び出しに応じて処理を分岐
9. Gradioにより、チャット、翻訳履歴、予約状況、音声再生、言語選択、音声入力を統合したUIを構築・起動

まとめ：  
このノートブックは「航空券の価格取得・予約ができる多言語対応のAIチャットボット」をGradioで構築し、音声入出力・翻訳・ツール連携も組み合わせた高度なプロトタイプを作成する内容です。

In [1]:
# Library for language translation
# Google翻訳を非公式API（ウェブスクレイピングベース）で利用するためAPIキーは不要
!pip install deep_translator

In [2]:
# Library for speech-to-text conversion
# make sure 'ffmpeg' is downloaded already
!pip install openai-whisper

In [3]:
# Library for storing and loading audio file
!pip install soundfile

In [4]:
# imports
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import base64
from io import BytesIO
from IPython.display import Audio, display
import tempfile
import whisper
import soundfile as sf

In [5]:
# 初期化

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')

if openai_api_key:
    print(f"OpenAI APIキーが存在し、開始します {openai_api_key[:8]}")
else:
    print("Openai APIキーが設定されていません")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

OpenAI APIキーが存在し、開始します sk-proj-


In [6]:
system_message = "あなたはFlightAIという航空会社の親切なアシスタントです。\
あなたの主な仕事は、お客様の疑問を解決し、航空券の価格を調べて予約することです。\
1文以内で、簡潔で丁寧な回答をしてください。常に正確に回答してください。\
わからない場合は、その旨を伝えてください。"

In [7]:
# Let's start by making a useful function

ticket_prices = {
    "london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499", "shanghai": "$799", "wuhan": "$899",
    "ロンドン": "$799", "パリ": "$899", "東京": "$1400", "ベルリン": "$499", "上海": "$799", "武漢": "$899",
}

# ticket_prices辞書から該当都市の価格を取得する関数
def get_ticket_price(destination_city):
    print(f"ツール get_ticket_price が {destination_city} に対して呼び出されました。")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

# 航空券の予約処理を行い、booked_citiesリストに追加、メッセージを返す関数
def book_ticket(destination_city):
    print(f"ツール book_ticket は {destination_city} に対して呼び出されました。")
    city = destination_city.lower()
    # global booked_cities
    if city in ticket_prices:
        price = ticket_prices.get(city, "")
        label = f"{city.title()} ({price})"
        i = booked_cities_choices.index(city.lower().capitalize())
        booked_cities_choices[i] = label
        booked_cities.append(label)
        return f"{city.title()} の予約が {ticket_prices[city]} で確定しました。"
    else:
        return "チケット価格に都市が見つかりません。"

In [8]:
# 関数定義の辞書構造
price_function = {
    "name": "get_ticket_price",
    "description": "目的地までの往復航空券の料金を取得します。例えば、顧客から「この都市までの航空券はいくらですか？」と尋ねられたときなど、航空券の料金を知りたいときはいつでもこのメソッドを呼び出します。",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "顧客が旅行したい都市",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

book_function = {
    "name": "book_ticket",
    "description": "目的地の都市への往復航空券を予約します。ユーザーが「この都市への航空券を予約して」などと言った場合など、目的地の都市への航空券を予約したいときにいつでも呼び出します。",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "顧客が航空券を予約したい都市"
            }
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

# ツールのリストに含める
tools = [
    {"type": "function", "function": price_function},
    {"type": "function", "function": book_function}
]

In [9]:
#talker と speak の違いは、音声の出力方法（再生方法）と保存の有無

# 音声出力
from pydub import AudioSegment
from pydub.playback import play

def talker(message):
    response = openai.audio.speech.create(
        model="tts-1",
        voice="onyx",    # Also, try replacing onyx with alloy
        input=message)
    
    audio_stream = BytesIO(response.content)
    audio = AudioSegment.from_file(audio_stream, format="mp3")
    play(audio)

def speak(message):
    response = openai.audio.speech.create(
        model="tts-1",
        voice="onyx",
        input=message)

    audio_stream = BytesIO(response.content)
    output_filename = "output_audio.mp3"
    with open(output_filename, "wb") as f:
        f.write(audio_stream.read())

    # Play the generated audio
    display(Audio(output_filename, autoplay=True))

In [10]:
# 翻訳処理
from deep_translator import GoogleTranslator

# Available translation language
LANGUAGES = {
    "English": "en",
    "Mandarin Chinese": "zh-CN",
    "Hindi": "hi",
    "Spanish": "es",
    "Arabic": "ar",
    "Bengali": "bn",
    "Portuguese": "pt",
    "Russian": "ru",
    "Japanese": "ja",
    "German": "de"
}

def update_lang(choice):
    global target_lang
    target_lang = LANGUAGES.get(choice, "zh-CN") 

def translate_message(text, target_lang):
    if target_lang == "en":
        return text
    try:
        translator = GoogleTranslator(source='auto', target=target_lang)
        return translator.translate(text)
    except:
        return f"Translation error: {text}"

In [11]:
# チャットボットの履歴からドロップダウンオプションを更新
def update_options(history):
    options = [f"{msg['role']}: {msg['content']}" for msg in history]
    return gr.update(choices=options, value=options[-1] if options else "")

# 選択したエントリからテキストコンテンツのみを抽出
def extract_text(selected_option):
    return selected_option.split(": ", 1)[1] if ": " in selected_option else selected_option

In [12]:
# オーディオ入力を numpy 配列として処理し、更新されたチャット履歴を返す
def speak_send(audio_np, history):
    if audio_np is None:
        return history

    # NumPyオーディオをメモリ内の.wavファイルに変換する
    sample_rate, audio_array = audio_np
    with tempfile.NamedTemporaryFile(suffix=".wav") as f:
        sf.write(f.name, audio_array, sample_rate)
        result = model.transcribe(f.name)
        text = result["text"]
        
    history += [{"role":"user", "content":text}]

    return None, history

In [13]:
# handle_tool_call 関数を記述
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    tool_name = tool_call.function.name
    arguments = json.loads(tool_call.function.arguments)

    if tool_name == "get_ticket_price":
        city = arguments.get("destination_city")
        price = get_ticket_price(city)
        response = {
            "role": "tool",
            "content": json.dumps({"destination_city": city,"price": price}),
            "tool_call_id": tool_call.id
        }
        return response, city

    elif tool_name == "book_ticket":
        city = arguments.get("destination_city")
        result = book_ticket(city)
        response = {
            "role": "tool",
            "content": result,
            "tool_call_id": tool_call.id            
        }
        return response, city

    else:
        return {
            "role": "tool",
            "content": f"No tool handler for {tool_name}",
            "tool_call_id": tool_call.id
        }, None


In [14]:
# The advanced 'chat' function in 'day5'
def interact(history, translated_history):
    messages = [{"role": "system", "content": system_message}] + history
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    
    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
        
    reply = response.choices[0].message.content
    translated_message = translate_message(history[-1]["content"], target_lang)
    translated_reply = translate_message(reply, target_lang)
    
    history += [{"role":"assistant", "content":reply}]
    translated_history += [{"role":"user", "content":translated_message}]
    translated_history += [{"role":"assistant", "content":translated_reply}]
    
    # Comment out or delete the next line if you'd rather skip Audio for now..
    talker(reply)

    return history, update_options(history), history, translated_history, update_options(translated_history), translated_history, gr.update(choices=booked_cities_choices, value=booked_cities)


In [15]:
with gr.Blocks() as demo:
    target_lang = "zh-CN"
    history_state = gr.State([]) 
    translated_history_state = gr.State([])
    booked_cities_choices = [key.lower().capitalize() for key in ticket_prices.keys()]
    booked_cities = []
    model = whisper.load_model("base")

    with gr.Row():
        city_checklist = gr.CheckboxGroup(
            label="Booked Cities",
            choices=booked_cities_choices     
        )
            
    with gr.Row():
        with gr.Column():
            chatbot = gr.Chatbot(label="Chat History", type="messages")
            selected_msg = gr.Dropdown(label="Select message to speak", choices=[])
            speak_btn = gr.Button("Speak")

        with gr.Column():
            translated_chatbot = gr.Chatbot(label="Translated Chat History", type="messages")
            translated_selected_msg = gr.Dropdown(label="Select message to speak", choices=[], interactive=True)
            translated_speak_btn = gr.Button("Speak")
    
    with gr.Row():
        language_dropdown = gr.Dropdown(
                choices=list(LANGUAGES.keys()),
                value="Mandarin Chinese",
                label="Translation Language",
                interactive=True
            )
      
    with gr.Row():
        entry = gr.Textbox(label="Chat with our AI Assistant:")

    with gr.Row():
        audio_input = gr.Audio(sources="microphone", type="numpy", label="Speak with our AI Assistant:")
    with gr.Row():
        audio_submit = gr.Button("Send")
    
    def do_entry(message, history):
        history += [{"role":"user", "content":message}]
        return "", history
        
    language_dropdown.change(fn=update_lang, inputs=[language_dropdown])

    speak_btn.click(
        lambda selected: speak(extract_text(selected)),
        inputs=selected_msg,
        outputs=None
    )

    translated_speak_btn.click(
        lambda selected: speak(extract_text(selected)),
        inputs=translated_selected_msg,
        outputs=None
    )

    entry.submit(do_entry, inputs=[entry, history_state], outputs=[entry, chatbot]).then(
        interact, inputs=[chatbot, translated_chatbot], outputs=[chatbot, selected_msg, history_state, translated_chatbot, translated_selected_msg, translated_history_state, city_checklist]
    )
    
    audio_submit.click(speak_send, inputs=[audio_input, history_state], outputs=[audio_input, chatbot]).then(
        interact, inputs=[chatbot, translated_chatbot], outputs=[chatbot, selected_msg, history_state, translated_chatbot, translated_selected_msg, translated_history_state, city_checklist]
    )
    # clear.click(lambda: None, inputs=None, outputs=chatbot, queue=False)

demo.launch()

* Running on local URL:  http://127.0.0.1:7867
* To create a public link, set `share=True` in `launch()`.


/home/seigi/.python3_venv/lib/python3.12/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


ツール get_ticket_price が ロンドン に対して呼び出されました。


Input #0, wav, from '/tmp/tmpd9xkyk8_.wav':   0KB sq=    0B f=0/0   
  Duration: 00:00:05.86, bitrate: 384 kb/s
  Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 24000 Hz, 1 channels, s16, 384 kb/s
   5.72 M-A: -0.000 fd=   0 aq=    0KB vq=    0KB sq=    0B f=0/0   

/home/seigi/.python3_venv/lib/python3.12/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


ツール book_ticket は ロンドン に対して呼び出されました。


Input #0, wav, from '/tmp/tmpbl639kfi.wav':   0KB sq=    0B f=0/0   
  Duration: 00:00:07.54, bitrate: 384 kb/s
  Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 24000 Hz, 1 channels, s16, 384 kb/s
   7.47 M-A: -0.000 fd=   0 aq=    0KB vq=    0KB sq=    0B f=0/0   

# 上記のUIの使い方

- 録音して[SEND]を押下すると、話した内容がChatに入力される。
- 行きたい都市の名前を行ってチケットの価格を確認する。
- 予約しますか？に「ハイ」と答えると予約が行われる。